<a href="https://colab.research.google.com/github/RubberLanding/AdversarialMachineLearning24/blob/evaluation/adversarial_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preliminaries


Install all needed packages.

In [7]:
!pip install torch
!pip install torchvision
# !pip install git+https://github.com/fra31/auto-attack
# !git pull https://github.com/RubberLanding/AdversarialMachineLearning24

Download the weights for the pre-trained Resnet18 on CIFAR-10. We only do this once and store them on a private GoogleDrive, which we later import to be able to actually load the weights.

In [2]:
# """Download pre-trained weights for ResNet18 on CIFAR10"""
# !pip install gdown

# # Source: https://github.com/huyvnphan/PyTorch_CIFAR10
# FILE_ID = "17fmN8eQdLpq2jIMQ_X0IXDPXfI9oVWgq"
# file_url = f"https://drive.google.com/uc?id={FILE_ID}"

# !gdown {file_url}

Import all packages und functions we need.

In [9]:

import numpy as np
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18
from torchvision.transforms import ToTensor
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts

RuntimeError: Only a single TORCH_LIBRARY can be used to register the namespace prims; please put all of your definitions in a single TORCH_LIBRARY block.  If you were trying to specify implementations, consider using TORCH_LIBRARY_IMPL (which can be duplicated).  If you really intended to define operators for a single namespace in a distributed way, you can use TORCH_LIBRARY_FRAGMENT to explicitly indicate this.  Previous registration of TORCH_LIBRARY was registered at /dev/null:241; latest registration was registered at /dev/null:241

Mount your GoogleDrive where you stored the weights for the pre-trained Resnet18 before. We will also store the logging files and model weights there.

In [ ]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')
project_dir = Path('/content/drive/MyDrive/adversarial_training')
project_dir.mkdir(parents=True, exist_ok=True)

import zipfile
import shutil

weight_dir = project_dir / "weights"
weight_dir.mkdir(parents=True, exist_ok=True)
weight_file = weight_dir / "resnet18.pt"

# """Extract the pre-trained model weights to Google Drive"""
# with zipfile.ZipFile("state_dicts.zip", "r") as zip_ref:
#   # print(zip_ref.namelist())
#   with zip_ref.open("state_dicts/resnet18.pt") as zf, open(weight_file, 'wb') as f:
#       shutil.copyfileobj(zf, f)

Define a convenience function to name the different training runs.

In [ ]:
import random
from datetime import datetime

def generate_run_name():
    """Generate a random name for a run."""
    colors = [
        "red", "blue", "green", "yellow", "purple", "orange", "pink",
        "black", "white", "gray", "silver", "gold", "cyan", "magenta"]
    adjectives = [
        "fast", "slow", "shiny", "dull", "bright", "dark", "silent",
        "loud", "brave", "calm", "wise", "fierce", "kind", "strong"]
    nouns = [
        "dragon", "tiger", "lion", "panda", "wolf", "phoenix", "eagle",
        "fox", "bear", "shark", "hawk", "cheetah", "whale", "octopus"]
    color = random.choice(colors)
    adjective = random.choice(adjectives)
    noun = random.choice(nouns)

    timestamp = datetime.now().strftime("%Y%m%d-%H%M")
    run_name = f"{color}-{adjective}-{noun}-{timestamp}"
    return run_name

# Data
Download the CIFAR-10 data. Split it into training, validation and test set. Do some pre-processing.

In [ ]:
batch_size = 1024 # batch size has to be < 2**16, should be <= 2**13 for T4
debug = True

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.491, 0.482, 0.446], std=[0.247, 0.243, 0.261]),
    ])

""" Load data """
data_train = CIFAR10(root="datasets", train=True, download=True, transform=transform)
data_test = CIFAR10(root="datasets", train=False, download=True, transform=transform)
data_test, data_val = torch.utils.data.random_split(data_test, [0.1, 0.9])

dataloader_train = DataLoader(data_train, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(data_test, batch_size=min(batch_size, len(data_test)), shuffle=False) # create test dataloader with a single batch
dataloader_val = DataLoader(data_val, batch_size=batch_size, shuffle=False)

num_classes = len(data_train.classes)

# mean = data_train.data.mean(axis=(0,1,2)) / 255 # [0.49139968, 0.48215841, 0.44653091]
# std = data_train.data.std(axis=(0,1,2)) / 255 # [0.24703223, 0.24348513, 0.26158784]

data_train_subset = Subset(data_train, list(range(2*batch_size)))
data_val_subset = Subset(data_val, list(range(batch_size)))
data_test_subset = Subset(data_test, list(range(100)))

dataloader_train_subset = DataLoader(data_train_subset, batch_size=batch_size, shuffle=True)
dataloader_val_subset = DataLoader(data_val_subset, batch_size=batch_size, shuffle=False)
dataloader_test_subset = DataLoader(data_test_subset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Attacks
Define the attacks.

In [ ]:
import torch.nn.functional as F

def fgsm(model, X, y, epsilon=8/255):
    """ Construct FGSM adversarial examples on the examples X"""
    delta = torch.zeros_like(X, requires_grad=True)
    loss = CrossEntropyLoss()(model(X + delta), y)
    loss.backward()
    return epsilon * delta.grad.detach().sign()

def pgd_linf(model, X, y, epsilon=8/255, alpha=2/255, num_iter=10, randomize=False):
    """ Construct FGSM adversarial examples on the examples X"""
    if randomize:
        delta = torch.rand_like(X, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon
    else:
        delta = torch.zeros_like(X, requires_grad=True)

    for t in range(num_iter):
        loss = CrossEntropyLoss()(model(X + delta), y)
        loss.backward()
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
        delta.grad.zero_()
    return delta.detach()

def pgd_linf_trades(model, X, y, epsilon=8/255, alpha=2/255, num_iter=10, randomize=False):
    """ Construct FGSM adversarial examples with KL-Divergence for TRADES.
        Should be used together with traded_loss().
    """
    if randomize:
        delta = torch.rand_like(X, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon
    else:
        delta = torch.zeros_like(X, requires_grad=True)

    for t in range(num_iter):
        # maybe set log_target=True and pass F.log_softmax(model(X), dim=1),
        # see the docs for more details
        loss = F.kl_div(F.log_softmax(model(X + delta), dim=1),
                        F.softmax(model(X), dim=1),
                        reduction='batchmean')
        loss.backward()
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
        delta.grad.zero_()
    return delta.detach()


# Losses
We define a wrapper, that calls the appropriate loss function with the correct arguments. Inside the wrapper, we implement the loss functions.

In [ ]:
class LossWrapper:
    def __init__(self, loss_fn, lambda_tradeoff=1.0):
        self.loss_fn = loss_fn
        self.lambda_tradeoff = lambda_tradeoff

    def __call__(self, model, X, y, delta=0.0):
        """Args:
              model: The model being trained.
              X: Clean input data.
              delta: Perturbations applied to X.
              y: Ground-truth labels.
        """
        # Cross-Entropy Loss
        if self.loss_fn == "CE":
            yp = model(X + delta)
            return F.cross_entropy(yp, y)

        # TRADES Loss
        elif self.loss_fn == "TRADES":
            yp_adv = model(X + delta)
            yp_clean = model(X)
            clean_loss = F.cross_entropy(yp_clean, y)
            robust_loss = F.kl_div(F.log_softmax(yp_adv, dim=1),
                            F.softmax(yp_clean, dim=1),
                            reduction='batchmean')
            return clean_loss + self.lambda_tradeoff * robust_loss

        else:
            raise ValueError("Unsupported loss function")

# Training

Define functions that train a model.

In [ ]:
def train_epoch(loader, model, opt, loss_fn):
    """Standard training/evaluation epoch over the dataset"""
    model.train()
    total_loss, total_err = 0.,0.
    for X,y in loader:
        X,y = X.to(device), y.to(device)
        yp = model(X)
        loss = loss_fn(model, X, y)
        opt.zero_grad()
        loss.backward()
        opt.step()

        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

@torch.no_grad()
def eval_epoch(loader, model, loss_fn):
    """Standard training/evaluation epoch over the dataset"""
    model.eval()
    total_loss, total_err = 0.,0.
    for X,y in loader:
        X,y = X.to(device), y.to(device)
        yp = model(X)
        loss = loss_fn(model, X, y)
        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

def train_epoch_adversarial(loader, model, attack, opt, loss_fn, **kwargs):
    """Adversarial training/evaluation epoch over the dataset"""
    model.train()
    total_loss, total_err = 0.,0.
    for X,y in loader:
        X,y = X.to(device), y.to(device)
        delta = attack(model, X, y, **kwargs)
        yp = model(X+delta)
        loss = loss_fn(model, X, y, delta)
        opt.zero_grad()
        loss.backward()
        opt.step()

        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

def eval_epoch_adversarial(loader, model, attack, loss_fn, **kwargs):
    """Adversarial training/evaluation epoch over the dataset"""
    model.eval()  # Set the model to evaluation mode
    total_loss, total_err = 0.0, 0.0

    for X, y in loader:
        X, y = X.to(device), y.to(device)

        # Compute adversarial perturbations (requires gradients)
        with torch.enable_grad():
            delta = attack(model, X, y, **kwargs)

        # Evaluate the model on adversarial examples without gradients
        with torch.no_grad():
            yp = model(X + delta)
            loss = loss_fn(model, X, y, delta)

            total_err += (yp.max(dim=1)[1] != y).sum().item()
            total_loss += loss.item() * X.shape[0]

    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

# Models

## Setup the Advarsarial Training Model


In [ ]:
import json
from torch.nn import CrossEntropyLoss, Conv2d

###########################
# LOAD THE MODEL

model_adv = resnet18()

# CIFAR10: kernel_size 7 -> 3, stride 2 -> 1, padding 3->1
model_adv.conv1 = Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model_adv.fc = torch.nn.Linear(model_adv.fc.in_features, num_classes)

pretrained_weights = torch.load(weight_file, weights_only=True)
model_adv.load_state_dict(pretrained_weights)
model_adv = model_adv.to(device)

###########################
# SET LOGGING

run_dir = project_dir / 'trades_loss'
run_dir.mkdir(parents=True, exist_ok=True)
log = {key: [] for key in ["train_losses", "test_losses", "adv_losses",
                           "train_errors", "test_errors", "adv_errors"]}


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/adversarial_training/adversarial_training/model_adv.pt'

## Train the Adversarial Training Model
Use adversarial training to train the robust model.

In [ ]:
###########################
# SET TRAINING PARAMETERS

opt = Adam(model_adv.parameters(), lr=1e-3, weight_decay=4e-3)
# opt = SGD(model_adv.parameters(), lr=1e-1)
# scheduler = CosineAnnealingLR(opt, T_max=100)
# scheduler = CosineAnnealingWarmRestarts(opt, T_0=10, T_mult=2, eta_min=0)

epochs = 30

trades = LossWrapper("TRADES")
cross_entropy = LossWrapper("CE")
###########################
# START TRAINING

print(f"Begin adversarial training run: {run_dir.stem}\n")
print(*("TR      ", "TE      ", "ADV     ", "Epoch   "), sep="\t")

for t in range(epochs):
    train_err, train_loss = train_epoch_adversarial(dataloader_train_subset, model_adv, pgd_linf_trades, opt, loss_fn=trades, randomize=True)
    test_err, test_loss = eval_epoch(dataloader_val_subset, model_adv, loss_fn=cross_entropy)
    adv_err, adv_loss = eval_epoch_adversarial(dataloader_val_subset, model_adv, fgsm, loss_fn=cross_entropy)

    # Update the losses and errors
    log["train_losses"] += [train_loss]
    log["test_losses"] += [test_loss]
    log["adv_losses"] += [adv_loss]
    log["train_errors"] += [train_err]
    log["test_errors"] += [test_err]
    log["adv_errors"] += [adv_err]

    print(*("{:.6f}".format(train_err),
            "{:.6f}".format(test_err),
            "{:.6f}".format(adv_err),
            f"{t+1}",), sep="\t")

###########################
# STORE RESULTS
store = True # set this variable to True when you have runs that you want to save
if store:
  with open(run_dir / "log.json", "w") as f:
      json.dump(log, f)
  torch.save(model_adv.state_dict(), run_dir / "model_adv.pt")

Begin adversarial training run: trades_loss

TR      	TE      	ADV     	Epoch   
0.553711	0.903320	0.919922	1
0.615723	0.594727	0.738281	2
0.503906	0.490234	0.655273	3
0.452148	0.453125	0.644531	4
0.410156	0.411133	0.599609	5
0.370605	0.375977	0.565430	6
0.324219	0.358398	0.554688	7
0.295410	0.336914	0.553711	8
0.249023	0.333984	0.534180	9
0.197266	0.325195	0.553711	10
0.183594	0.326172	0.556641	11
0.146973	0.326172	0.539062	12
0.116699	0.313477	0.547852	13
0.081543	0.329102	0.565430	14
0.064453	0.307617	0.543945	15
0.047852	0.303711	0.556641	16
0.029785	0.333008	0.553711	17
0.029297	0.343750	0.563477	18
0.023926	0.313477	0.531250	19
0.013184	0.329102	0.541016	20
0.021484	0.328125	0.538086	21
0.022461	0.358398	0.533203	22
0.031250	0.324219	0.516602	23
0.009766	0.346680	0.547852	24
0.024414	0.324219	0.531250	25
0.006836	0.308594	0.522461	26
0.010254	0.310547	0.516602	27
0.005371	0.305664	0.506836	28
0.008301	0.310547	0.507812	29
0.005371	0.316406	0.527344	30


Install AutoAttack to calculate a robust accuracy for the model to get a fair comparison, e.g. with models from [RobustBench](https://robustbench.github.io/).

In [5]:
!pip install git+https://github.com/fra31/auto-attack

  Cloning https://github.com/fra31/auto-attack to /tmp/pip-req-build-_zt7toza
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack /tmp/pip-req-build-_zt7toza
  Resolved https://github.com/fra31/auto-attack to commit a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Preparing metadata (setup.py) ... done
  Created wheel for autoattack: filename=autoattack-0.1-py3-none-any.whl size=36229 sha256=51f95e92f74f6a9f98b12157b543f4699a61fee88c816529e8190e7543eb9d24
  Stored in directory: /tmp/pip-ephem-wheel-cache-7ll2ry5m/wheels/b7/17/95/4b16fafe1b84fdabd247eb5f01b41165bd98d4c82d64ab93d2
Successfully built autoattack


In [ ]:
from autoattack import AutoAttack

X, y = next(iter(dataloader_test)) # optimally, the test loader has a single batch containing the entire test set
X, y = X.to(device), y.to(device)

adversary = AutoAttack(model_adv, norm='Linf', eps=8/255, version='standard')
adv_examples = adversary.run_standard_evaluation(X, y, bs=batch_size) # this takes ~8min on T4 on a single test batch

## Setup the Weight Averaging Model
Here we initialize an exponential moving average (EMA) model, based on the robust model.

In [ ]:
import json
from torch.nn import CrossEntropyLoss, Conv2d

###########################
# LOAD THE MODEL

model_adv = resnet18()

# CIFAR10: kernel_size 7 -> 3, stride 2 -> 1, padding 3->1
model_adv.conv1 = Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model_adv.fc = torch.nn.Linear(model_adv.fc.in_features, num_classes)

pretrained_weights = torch.load(weight_file, weights_only=True)
model_adv.load_state_dict(pretrained_weights)

model_ema = torch.optim.swa_utils.AveragedModel(model_adv, multi_avg_fn=torch.optim.swa_utils.get_ema_multi_avg_fn(0.9))

model_adv = model_adv.to(device)
model_ema = model_ema.to(device)

###########################
# SET LOGGING

run_dir = project_dir / 'weight_averaging'
run_dir.mkdir(parents=True, exist_ok=True)
log = {key: [] for key in ["train_losses", "test_losses", "adv_losses",
                           "train_errors", "test_errors", "adv_errors"]}


# Train the Weight Averaged Model

In [ ]:
###########################
# SET TRAINING PARAMETERS

opt = Adam(model_adv.parameters(), lr=1e-3, weight_decay=4e-3)
# opt = SGD(model_adv.parameters(), lr=1e-1)
# scheduler = CosineAnnealingLR(opt, T_max=100)
# scheduler = CosineAnnealingWarmRestarts(opt, T_0=10, T_mult=2, eta_min=0)

epochs = 30

trades = LossWrapper("TRADES")
cross_entropy = LossWrapper("CE")

###########################
# START TRAINING

print(f"Begin adversarial training run: {run_dir.stem}\n")
print(*("TR      ", "TE      ", "ADV     ", "Epoch   "), sep="\t")

for t in range(epochs):
    train_err, train_loss = train_epoch_adversarial(dataloader_train_subset, model_adv, pgd_linf, opt, loss_fn=cross_entropy, randomize=True)
    model_ema.update_parameters(model_adv) # Update EMA model

    test_err, test_loss = eval_epoch(dataloader_val_subset, model_ema, loss_fn=cross_entropy) # Evaluate clean acc. on EMA model
    adv_err, adv_loss = eval_epoch_adversarial(dataloader_val_subset, model_ema, fgsm, loss_fn=cross_entropy) # Evaluate robust acc. on EMA model

    # Update the losses and errors
    log["train_losses"] += [train_loss]
    log["test_losses"] += [test_loss]
    log["adv_losses"] += [adv_loss]
    log["train_errors"] += [train_err]
    log["test_errors"] += [test_err]
    log["adv_errors"] += [adv_err]

    print(*("{:.6f}".format(train_err),
            "{:.6f}".format(test_err),
            "{:.6f}".format(adv_err),
            f"{t+1}",), sep="\t")

###########################
# STORE RESULTS
store = False # set this variable to True when you have runs that you want to save
if store:
  with open(run_dir / "log.json", "w") as f:
      json.dump(log, f)
  torch.save(model_adv.state_dict(), run_dir / "model_adv.pt")

Begin adversarial training run: weight_averaging

TR      	TE      	ADV     	Epoch   
0.752930	0.890625	0.897461	1
0.795410	0.725586	0.802734	2
0.688965	0.882812	0.892578	3
0.656250	0.891602	0.899414	4
0.634766	0.901367	0.901367	5
0.602051	0.901367	0.901367	6
0.576660	0.901367	0.901367	7
0.562012	0.900391	0.901367	8
0.538086	0.881836	0.889648	9
0.521973	0.809570	0.838867	10
0.502930	0.740234	0.783203	11
0.469727	0.692383	0.742188	12
0.452637	0.687500	0.728516	13
0.415527	0.671875	0.725586	14
0.384766	0.652344	0.715820	15
0.358887	0.602539	0.681641	16
0.334961	0.575195	0.660156	17
0.293457	0.517578	0.620117	18
0.267578	0.516602	0.617188	19
0.248047	0.486328	0.590820	20
0.213867	0.474609	0.587891	21
0.173828	0.470703	0.583008	22
0.167480	0.479492	0.583984	23
0.160645	0.467773	0.591797	24
0.117676	0.458008	0.574219	25
0.131348	0.442383	0.571289	26
0.102539	0.440430	0.570312	27
0.065430	0.434570	0.557617	28
0.037598	0.413086	0.551758	29
0.033203	0.426758	0.569336	30


In [ ]:
#

In [4]:
# Evaluation Functions

import json
import matplotlib.pyplot as plt
import io
import contextlib
from autoattack import AutoAttack

# Sanity Check: Loading the log file and plotting adversarial accuracies over training epochs
def sanity_check(model):
  project_dir = Path('/content/MyDrive/adversarial_training')
  with open(project_dir / model / "log.json", "r") as f:
      log = json.load(f)

  plt.plot(log["adv_errors"])
  plt.xlabel("Epoch")
  plt.ylabel("Robust Accuracy")
  return None

# Evaluate Baseline Robust Accuracy and Final Robust Accuracy (after training) of a Model
def evaluate_robust_accuracy(model, dataloader, attack):
    model = resnet18()
    model.conv1 = Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.fc = torch.nn.Linear(model_adv.fc.in_features, num_classes)
    model.eval()
    model = model.to(device)
    baseline_robust_accuracy = 1 - eval_epoch_adversarial(dataloader, model, attack, loss_fn=cross_entropy)[0]

    # load weight file to evaluate trained model
    model.load_state_dict(torch.load(weight_file, weights_only=True))
    model = model.to(device)
    model.eval()
    final_robust_accuracy = 1 - eval_epoch_adversarial(dataloader, model, attack, loss_fn=cross_entropy)[0]
    print(f"Baseline Robust Accuracy: {baseline_robust_accuracy}")
    print(f"Final Robust Accuracy: {final_robust_accuracy}")
    return baseline_robust_accuracy, final_robust_accuracy

def evaluate(weight_file, dataloader, attack):
    model = resnet18()
    model.conv1 = Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.fc = torch.nn.Linear(model_adv.fc.in_features, num_classes)
    model.eval()
    model.load_state_dict(torch.load(weight_file, weights_only=True))
    model = model.to(device)
    clean_accuracy = 1 - eval_epoch(dataloader, model, loss_fn=cross_entropy)[0]
    if attack == 'fgsm':
        robust_accuracy = 1 - eval_epoch_adversarial(dataloader, model, fgsm, loss_fn=cross_entropy)[0]
    elif attack == 'pgd':
      robust_accuracy = 1 - eval_epoch_adversarial(dataloader, model, pgd_linf, loss_fn=cross_entropy)[0]
    elif attack == 'autoattack':
        x_test, y_test = next(iter(dataloader))
        x_test, y_test = x_test.to(device), y_test.to(device)
        output_buffer = io.StringIO()
        with contextlib.redirect_stdout(output_buffer):
            adversary = AutoAttack(model_adv, norm='Linf', eps=8/255, version='standard', device=device)
            adversary.run_standard_evaluation(x_test, y_test, bs=128)

        output = output_buffer.getvalue()

        lines = output.splitlines()
        for line in reversed(lines):
            if "robust accuracy:" in line.lower():
                robust_acc = float(line.split(":")[-1].strip().replace("%", "")) / 100.0
                break
        else:
            robust_acc = None
    return clean_accuracy, robust_accuracy


ModuleNotFoundError: No module named 'autoattack'

In [ ]:
# Pareto plot of all models

models_to_evaluate = ['adversarial_training','trades_loss','weight_averaging']
clean_accuracies = []
robust_accuracies = []
# for plotting
handles = []

for model in models_to_evaluate:
    weight_file = project_dir / model / "model_adv.pt"
    attack = 'fgsm'
    dataloader_test = dataloader_test_subset

    clean_accuracy, robust_accuracy = evaluate(weight_file, dataloader_test, attack)
    clean_accuracies.append(clean_accuracy)
    robust_accuracies.append(robust_accuracy)
    handle = plt.scatter(clean_accuracy, robust_accuracy, label=model)
    handles.append(handle)

plt.xlabel('Clean Accuracy')
plt.ylabel('Robust Accuracy')
plt.title('Pareto Plot of Models')
plt.legend(handles=handles)
plt.grid(True)
plt.show()

In [ ]:
""" Regular Training """
import json
from torch.nn import CrossEntropyLoss, Conv2d

model_reg = resnet18()

# CIFAR10: kernel_size 7 -> 3, stride 2 -> 1, padding 3->1
model_reg.conv1 = Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model_reg.fc = torch.nn.Linear(model_reg.fc.in_features, num_classes)

pretrained_weights = torch.load(weight_file, weights_only=True)
model_reg.load_state_dict(pretrained_weights)
model_reg = model_reg.to(device)

run_dir = project_dir / generate_run_name()
run_dir.mkdir(parents=True, exist_ok=True)

opt = SGD(model_reg.parameters(), lr=1e-1)
opt = Adam(model_reg.parameters(), lr=1e-3)

epochs = 2
log = {key: [] for key in ["train_losses", "test_losses", "adv_losses",
                           "train_errors", "test_errors", "adv_errors"]}

print(f"Begin adversarial training run: {run_dir.stem}\n")
print(*("TR      ", "TE      ", "ADV     ", "     "), sep="\t")

for t in range(epochs):
    train_err, train_loss = train_epoch(dataloader_train, model_reg, opt)
    test_err, test_loss = eval_epoch(dataloader_test, model_reg)
    adv_err, adv_loss = eval_epoch_adversarial(dataloader_test, model_reg, fgsm)

    # Update the losses and errors
    log["train_losses"] += [train_loss]
    log["test_losses"] += [test_loss]
    log["adv_losses"] += [adv_loss]
    log["train_errors"] += [train_err]
    log["test_errors"] += [test_err]
    log["adv_errors"] += [adv_err]

    print(*("{:.6f}".format(train_err),
            "{:.6f}".format(test_err),
            "{:.6f}".format(adv_err),
            f"Epoch: {t+1}",), sep="\t")

with open(run_dir / "log.json", "w") as f:
    json.dump(log, f)
torch.save(model_reg.state_dict(), run_dir / "model_reg.pt")

In [ ]:
"""
ADV evaluated with FGSM
We observe that the ADV validation error is lower than the training error because FGSM is a weaker attack thatn PGD
Begin adversarial training run: magenta-slow-phoenix-20250105-1507

TR      	TE      	ADV     	Epoch
0.666760	0.406778	0.493000	1
0.539120	0.314444	0.405000	2
0.476300	0.274778	0.379000	3
0.432680	0.245222	0.339556	4
0.402620	0.230444	0.330000	5

ADV evaluated with PGD-Linf
We observe that the ADV validation error is higher than the training error as expected
Begin adversarial training run: orange-kind-shark-20250105-1531

TR      	TE      	ADV     	Epoch
0.675220	0.419667	0.599000	1
0.543500	0.310889	0.508778	2
0.472780	0.271000	0.475556	3
0.434380	0.248000	0.461889	4
0.403640	0.230444	0.457889	5
"""

In [ ]:
"""
gold-bright-eagle-20241204-1128:
SGD(lr=1e-1), batch_size=1024
"""
"""
orange-kind-hawk-20241204-1148:
SGD(lr=1e-1, weight_decay=5e-4), batch_size=1024
"""
"""
silver-shiny-phoenix-20241204-1210:
SGD(lr=1e-1, weight_decay=5e-4, momentum=0.9, nesterov=True), batch_size=1024
"""
"""
gray-fierce-octopus-20241204-1226:
SGD(lr=1e-1, weight_decay=5e-4, momentum=0.9, nesterov=True), batch_size=1024, CosineAnnealingLR
"""
"""
gold-loud-cheetah-20241204-1414:
SGD(lr=1e-1, weight_decay=5e-4, momentum=0.9, nesterov=True), batch_size=1024,, epsilon=4/255
"""
"""
cyan-wise-eagle-20241203-1710:
SGD(lr=1e-1, weight_decay=5e-4, momentum=0.9, nesterov=True), batch_size=1024,, epsilon=32/255
"""